In [11]:
import matplotlib
#matplotlib.use('qtagg')
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#
import matplotlib.pyplot as plt
import numpy as np
import h5py

import os

Autosaving every 180 seconds
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<ipython-input-11-e54c02c9d18a>:13: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [12]:
# function that loads the original datasets
def reload_names():

    # hard coded names for each session x 3 recordings
    fname_tracks = [

        ['/media/cat/4TB/donato/steffen/DON-004366/20210228/suite2p/plane0/mat/DON-004366_20210228_TRD-2P_S1-ACQ_eval.mat',
         '/media/cat/4TB/donato/steffen/DON-004366/20210228/suite2p/plane0/mat/DON-004366_20210228_TRD-2P_S2-ACQ_eval.mat',
         '/media/cat/4TB/donato/steffen/DON-004366/20210228/suite2p/plane0/mat/DON-004366_20210228_TRD-2P_S3-ACQ_eval.mat'],

         # 
         ['/media/cat/4TB/donato/steffen/DON-004366/20210301/mat/DON-004366_20210301_TRD-2P_S1-ACQ_eval.mat',
         '/media/cat/4TB/donato/steffen/DON-004366/20210301/mat/DON-004366_20210301_TRD-2P_S2-ACQ_eval.mat',
         '/media/cat/4TB/donato/steffen/DON-004366/20210301/mat/DON-004366_20210301_TRD-2P_S3-ACQ_eval.mat'],

        # 
        ['/media/cat/4TB/donato/steffen/DON-004366/20210303/DON-004366_20210303_TRD-2P_S1-ACQ_eval.mat',
        '/media/cat/4TB/donato/steffen/DON-004366/20210303/DON-004366_20210303_TRD-2P_S2-ACQ_eval.mat',
        '/media/cat/4TB/donato/steffen/DON-004366/20210303/DON-004366_20210303_TRD-2P_S3-ACQ_eval.mat'
    ]

    # 
    fnames_ca = [
        '/media/cat/4TB/donato/steffen/DON-004366/20210228/suite2p/plane0/binarized_traces/F_upphase.npy',
        '/media/cat/4TB/donato/steffen/DON-004366/20210301/binarized_traces/F_upphase.npy',
        '/media/cat/4TB/donato/steffen/DON-004366/20210303/binarized_traces/F_upphase.npy']

    # segmaentaiton: 
    segment_order = [
                    ["A", "B", "A'"],
                    ["A", "A'","B"],
                    ["A", "B", "A'"]       
                    ]
    
    return fname_tracks, fnames_ca, segment_order

#
def load_tracks(fname_tracks, bin_width=5):

    #
    pos_tracks = []
    idx_tracks = []
    idx_ctr = 0
    for fname_track in fname_tracks:
        with h5py.File(fname_track, 'r') as file:
            #
            pos = file['trdEval']['position_atframe'][()].squeeze()

            n_timesteps = pos.shape[0]
            #print ("# of time steps: ", pos.shape)

        # bin speed for some # of bins
        #bin_width = 5
        sum_flag = False
        pos_bin = run_binning(pos, bin_width, sum_flag)
        print ("pos bin: ", pos_bin.shape)

        #
        pos_tracks.append(pos_bin)

        # add locations of the belt realtive
        temp = np.arange(idx_ctr,idx_ctr+n_timesteps,1)
        idx_tracks.append(temp)
    
        idx_ctr+=n_timesteps
        
    return pos_tracks, idx_tracks

def run_binning(data, bin_size=7, sum_flag=True):
    
    # split data into bins
    idx = np.arange(0, data.shape[0], bin_size)
    d2 = np.array_split(data, idx[1:])
    
    # sum on time axis; drop last value
    if sum_flag:
        d3 = np.array(d2[:-1]).sum(1)
    else:
        d3 = np.median(np.array(d2[:-1]), axis=1)
    
    print ("   Data binned using ", bin_size, " frame bins, final size:  ", d3.shape)

    #    
    return d3


In [14]:
#######################################
###### 2D/3D GRAPHICAL PLOTS ###########
########################################
#
from tqdm import trange

#  
fname_tracks, fnames_ca, segment_order = reload_names()

#
len_seg_cm = 1       # discretize belt position into 6 ~equal sections
triage_value = 0.25  # remove xx% most outlier states
bin_width = 1        # population vector binning
sample_rate = 30
randomize = False

#
experiment_ids = [0,1,2]

#experiment_ids = [0]
session_ids = [0,1,2]

#
for experiment_id in experiment_ids:
    fname_tracks1 = fname_tracks[experiment_id]
    fname_ca1 = fnames_ca[experiment_id]
    segment_order1 = segment_order[experiment_id]

    #
    pos_tracks, idx_tracks = load_tracks(fname_tracks1, bin_width)

    # 
    for session_id in session_ids:
     
        print ("loading: ", experiment_id, session_id)

        #
        data_ca = np.load(fname_ca1)
        dimensionality = data_ca.shape[1]

        #
        root_dir = '/home/cat/'
        fname_base = os.path.join(root_dir, 
                                  #experiment_ids[experiment_id],
                                  os.path.split(fname_tracks1[session_id])[1].replace('.mat','_session_'+str(session_id)))
                                                                                    
        #
        np.save(fname_base+ '_ca_data.npy', data_ca[:,idx_tracks[session_id]])
        np.save(fname_base+ '_pos_track.npy', pos_tracks[session_id])
        np.save(fname_base+ '_idx_track.npy', idx_tracks[session_id])
                


print ("DONE")

   Data binned using  1  frame bins, final size:   (37159,)
pos bin:  (37159,)
   Data binned using  1  frame bins, final size:   (18579,)
pos bin:  (18579,)
   Data binned using  1  frame bins, final size:   (37159,)
pos bin:  (37159,)
loading:  0 0
loading:  0 1
loading:  0 2
   Data binned using  1  frame bins, final size:   (37159,)
pos bin:  (37159,)
   Data binned using  1  frame bins, final size:   (37159,)
pos bin:  (37159,)
   Data binned using  1  frame bins, final size:   (18579,)
pos bin:  (18579,)
loading:  1 0
loading:  1 1
loading:  1 2
   Data binned using  1  frame bins, final size:   (37159,)
pos bin:  (37159,)
   Data binned using  1  frame bins, final size:   (18579,)
pos bin:  (18579,)
   Data binned using  1  frame bins, final size:   (37159,)
pos bin:  (37159,)
loading:  2 0
loading:  2 1
loading:  2 2
DONE
